In [1]:
import json
import numpy as np
import pandas as pd

In [2]:
df_list = list()
for i in range(1, 6):
    list_path = 'scrapped/data'+str(i)+'.json'
    json_list = open(list_path)
    json_list = json.load(json_list)
    df = pd.DataFrame(json_list)
    df['p_windows'] = df['p_windows'].apply(lambda x: ' '.join(x) if isinstance(x, list) else x)
    df = df.loc[df['pprice'].notnull()]
    df_list.append(df)

In [3]:
df = pd.concat(df_list).drop_duplicates('id')

In [4]:
print(df.shape)
print(df.columns)

(14212, 54)
Index(['id', 'title', 'estate', 'isExclusiveOffer', 'isPrivateOwner',
       'isPromoted', 'areaInSquareMeters', 'terrainAreaInSquareMeters',
       'roomsNumber', 'hidePrice', 'dateCreated', 'dateCreatedFirst', 'loc',
       'loc_label', 'desc', 'price', 'url', 'agency_name', 'description',
       'features', 'm', 'price_per_m', 'rooms_num', 'market', 'building_type',
       'floor_no', 'building_floors_num', 'windows_type', 'heating',
       'build_year', 'construction_status', 'rent', 'building_ownership',
       'lat', 'lon', 'district', 'features2', 'p_kitchen', 'p_parking',
       'p_rooms', 'p_type', 'p_windows', 'p_btype', 'p_bmaterial',
       'p_bwindows', 'p_bheating', 'owner_type', 'owner_name', 'object_name',
       'pprice', 'remote_services', 'building_material', 'free_from',
       'property'],
      dtype='object')


In [5]:
pd.DataFrame(df.dtypes).transpose()

,id,title,estate,isExclusiveOffer,isPrivateOwner,isPromoted,areaInSquareMeters,terrainAreaInSquareMeters,roomsNumber,hidePrice,...,p_bwindows,p_bheating,owner_type,owner_name,object_name,pprice,remote_services,building_material,free_from,property
0,int64,object,object,bool,bool,bool,float64,object,object,bool,...,object,object,object,object,object,object,object,object,object,float64


In [6]:
df.select_dtypes(['object']).apply(lambda x: x.unique())

title                        [Mieszkanie 102m2 + Balkon 13.2m2 - Bezposredn...
estate                                                                  [FLAT]
terrainAreaInSquareMeters                                               [None]
roomsNumber                  [FOUR, THREE, TWO, FIVE, SIX, ONE, EIGHT, SEVE...
dateCreated                  [2023-06-25 19:59:39, 2023-06-25 19:51:42, 202...
dateCreatedFirst             [2023-04-26 19:18:02, 2023-05-03 19:49:18, 202...
loc                          [None, ul. Stanisława Kazury, , ul. Lucjana Ru...
loc_label                    [Warszawa, mazowieckie, Warszawa, Ursynów, ul....
desc                         [Świetne 4 pokojowe mieszkanie na sprzedaż w m...
url                          [https://www.otodom.pl/pl/oferta/mieszkanie-10...
agency_name                  [None, BJM Development, SLOW HOME Nieruchomośc...
description                  [\nNa sprzedaż słoneczny, dwustronny i przestr...
features                     [internet, teren zamkni

In [7]:
df.select_dtypes(['bool']).apply(lambda x: x.unique())

,isExclusiveOffer,isPrivateOwner,isPromoted,hidePrice
0,False,True,True,False
1,True,False,False,True


In [8]:
df.describe()

,id,areaInSquareMeters,price,lat,lon,property
count,1.421200e+04,14212.000000,1.012400e+04,14212.000000,14212.000000,0.0
mean,6.387401e+07,64.947562,1.078417e+06,52.230892,21.002221,NaN
std,6.673876e+05,151.264762,1.166693e+06,0.055283,0.093568,NaN
min,4.359478e+07,8.000000,6.000000e+03,50.044050,16.066580,NaN
25%,6.373878e+07,43.790000,5.950000e+05,52.198320,20.951830,NaN
50%,6.412275e+07,56.390000,7.650000e+05,52.228569,21.002420,NaN
75%,6.426826e+07,71.000000,1.147141e+06,52.258812,21.053113,NaN
max,6.435696e+07,17500.000000,2.135010e+07,52.363830,21.309846,NaN


In [9]:
df['is_kitchen_separate'] = np.where(df['p_kitchen'] == 'SEPARATE', 1, 0)

cols_to_drop = [
    'features2', 'p_kitchen', 'p_parking', 'p_rooms', 'p_type', 'p_windows',
    'estate', 'terrainAreaInSquareMeters', 'roomsNumber', 'url'
]

df = df.drop(columns=cols_to_drop)

In [10]:
df.select_dtypes(['object']).apply(lambda x: x.unique())

title                  [Mieszkanie 102m2 + Balkon 13.2m2 - Bezposredn...
dateCreated            [2023-06-25 19:59:39, 2023-06-25 19:51:42, 202...
dateCreatedFirst       [2023-04-26 19:18:02, 2023-05-03 19:49:18, 202...
loc                    [None, ul. Stanisława Kazury, , ul. Lucjana Ru...
loc_label              [Warszawa, mazowieckie, Warszawa, Ursynów, ul....
desc                   [Świetne 4 pokojowe mieszkanie na sprzedaż w m...
agency_name            [None, BJM Development, SLOW HOME Nieruchomośc...
description            [\nNa sprzedaż słoneczny, dwustronny i przestr...
features               [internet, teren zamknięty, domofon / wideofon...
m                      [102, 64.9, 52, 35, 125, 55, 95, 71.37, 158, 9...
price_per_m            [11755, 14022, 15288, 14143, 15120, 14891, 181...
rooms_num                             [4, 3, 2, 5, 6, 1, 8, 7, more, 10]
market                                              [secondary, primary]
building_type          [apartment, block, nan, hous

In [11]:
df.to_csv('data/flats.csv')